#### mega.nz auth data

In [0]:
mega_login = "ivanovaek21@gmail.com"
mega_password = "etobol234Q"

#### languages to learn translate between

In [0]:
langpair_to_learn_pair = ("kaz", "tur")
langpair_to_learn = '-'.join(langpair_to_learn_pair)

#### install MEGAcmd

In [0]:
!rm -r sample_data/
!wget https://mega.nz/linux/MEGAsync/xUbuntu_18.04/amd64/megacmd-xUbuntu_18.04_amd64.deb
!sudo apt install libc-ares2 libcrypto++6 libmediainfo0v5 libzen0v5
!sudo apt --fix-broken install
!sudo dpkg -i megacmd-xUbuntu_18.04_amd64.deb

!nohup mega-login {mega_login} {mega_password} && echo $! > tmp.pid && sleep 2
tmp_pid = !cat tmp.pid
!kill {tmp_pid}
!mega-login {mega_login} {mega_password}
!echo f"{langpair_to_learn}\n" > foo.txt
!mega-put foo.txt -c /aligned-embeddings/{langpair_to_learn}/

--2019-02-17 16:29:26--  https://mega.nz/linux/MEGAsync/xUbuntu_18.04/amd64/megacmd-xUbuntu_18.04_amd64.deb
Resolving mega.nz (mega.nz)... 89.44.169.135, 31.216.148.10, 2001:678:25c:2215::553, ...
Connecting to mega.nz (mega.nz)|89.44.169.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6779936 (6.5M) [application/x-deb]
Saving to: ‘megacmd-xUbuntu_18.04_amd64.deb’

megacmd-xUbuntu_18. 100%[===================>]   6.47M  --.-KB/s    in 0.1s    

2019-02-17 16:29:26 (52.9 MB/s) - ‘megacmd-xUbuntu_18.04_amd64.deb’ saved [6779936/6779936]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmms0 libtinyxml2-6
The following NEW packages will be installed:
  libc-ares2 libcrypto++6 libmediainfo0v5 libmms0 libtinyxml2-6 libzen0v5
0 upgraded, 6 newly installed, 0 to remove and 8 not upgraded.
Need to get 2,881 kB of archives.
After this operation, 10.6 MB of addit

#### provide mapping between var-dial and fasstext language codes

In [0]:
vardial2fasttext = {
    "ast":"ast",
    "cat":"ca",
    "fra":"fr",
    "ita":"it",
    "por":"pt",
    "spa":"es",
    "bak":"ba",
    "crh":"crh",
    "kaz":"kk",
    "kir":"ky",
    "tat":"tt",
    "tur":"tr"
}
fasttext2vardial = {ft_code:vd_code for vd_code,ft_code in vardial2fasttext.items()}

langs_to_process = {
#     "roa": ("ast", "cat", "fra", "ita", "por", "spa"), 
    "trk": ("bak", "crh", "kaz", "kir", "tat", "tur")
}


#### get vectors

* download fasttext vectors for the task
* look how many task-words are not in the embeddings set
* do some nonneeded and slow vectors preprocessing

In [0]:

!mkdir vectors

for group_name, group_langs in langs_to_process.items():

    !wget -O train-{group_name}-uncovered -q https://raw.githubusercontent.com/ftyers/vardial-shared-task/master/train/{group_name}-uncovered
    !wget -O dev-{group_name}-uncovered -q https://raw.githubusercontent.com/ftyers/vardial-shared-task/master/dev/{group_name}-uncovered
    !wget -O dev-{group_name}-covered -q https://raw.githubusercontent.com/ftyers/vardial-shared-task/master/dev/{group_name}-covered

    !cat  train-{group_name}-uncovered >> {group_name}-alldata
    !cat  dev-{group_name}-uncovered >> {group_name}-alldata
    !cat  dev-{group_name}-covered >> {group_name}-alldata
    
    for lang_to_process in group_langs:
        lang_ft_code = vardial2fasttext[lang_to_process]
        !wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.{lang_ft_code}.vec
        
        
        alphabet = !cut -f2 {group_name}-alldata | grep -o . | sort -u
        for idx in range(len(alphabet)):
            elem = alphabet[idx]
            if elem in ['-','^',']']:
                alphabet[idx] = '\\'+elem
        alphabet = "".join(c for c in alphabet)
        
        vec_words_fn = f"{lang_ft_code}_vec_short.vec"
        tracked_words_fn = f"{lang_to_process}_tracked"
        
        get_ipython().system(f"grep -P \"^{lang_to_process}\" {group_name}-alldata | cut -f2 | sort -u > {tracked_words_fn}")
        
        cmd_to_exec = f"grep -P \"^[{alphabet}]+\s\" wiki.{lang_ft_code}.vec | " + "grep -v -P \"^\w*\d\w*\" | cut -d$\' \' -f1 > " + f"{vec_words_fn}"
        get_ipython().system(cmd_to_exec)
        
        words_having_vectors_fn = f"{lang_to_process}_vectorized_words"
        words_non_having_vectors_fn = f"{lang_to_process}_nonvectorized_words"
        
        with open(tracked_words_fn, 'r', encoding='utf-8') as f_tracked,\
            open(vec_words_fn, 'r', encoding='utf-8') as f_vectors,\
            open(words_having_vectors_fn, 'w', encoding='utf-8') as f_vectorized,\
            open(words_non_having_vectors_fn, 'w', encoding='utf-8') as f_non_vectorized:
            
            track_words = set([line.strip() for line in f_tracked.readlines()])
            for line in f_vectors:
                line = line.strip()
                if line in track_words:
                    print(line, file=f_vectorized)
                else:
                    print(line, file=f_non_vectorized)
                    
        num_words_in_track = len(track_words)
        num_words_vectorized = !wc -l {words_having_vectors_fn} | cut -d$' ' -f1
        
        num_words_vectorized = int(num_words_vectorized[0])
        num_words_non_vectorized = num_words_in_track - num_words_vectorized
        
        print(", ".join([f"lang: {lang_to_process:>3}",
              f"words in track: {num_words_in_track:>6}",
              f"vectorized words num: {num_words_vectorized:>6}",
              f"non vectorized words num: {num_words_non_vectorized:>6}"]))
        
        tracked_vectors_fn = f"{lang_to_process}_tracked.vec"
        with open(f"wiki.{lang_ft_code}.vec", 'r', encoding="utf-8") as vectors_f, \
            open(tracked_vectors_fn, 'w', encoding="utf-8") as tracked_vectors_f:
                for line in vectors_f:
                    if line.strip().split()[0] in track_words:
                        print(line.strip(), file=tracked_vectors_f)
                        
            

lang: bak, words in track:   9999, vectorized words num:   9803, non vectorized words num:    196
lang: crh, words in track:    999, vectorized words num:    245, non vectorized words num:    754
lang: kaz, words in track:   9995, vectorized words num:   9928, non vectorized words num:     67
lang: kir, words in track:  10000, vectorized words num:   9938, non vectorized words num:     62
lang: tat, words in track:  10000, vectorized words num:   9864, non vectorized words num:    136
lang: tur, words in track:   9990, vectorized words num:   9897, non vectorized words num:     93


In [0]:
!mv *.vec vectors
!mkdir words
!mv *_* words


for langgroup in langs_to_process.keys():
    for lang in langs_to_process[langgroup]:
        !mkdir vectors/{lang}
        ftcode_of_lang = vardial2fasttext[lang]
        !mv vectors/{lang}*.vec vectors/{lang}
        !mv vectors/{ftcode_of_lang}_vec_short.vec vectors/{lang}/{lang}_vec_short.vec
        !mv vectors/wiki.{ftcode_of_lang}.vec vectors/{lang}/{lang}_wiki.vec

!ls vectors/*/

mv: cannot stat 'vectors/crh_vec_short.vec': No such file or directory
vectors/bak/:
bak_tracked.vec  bak_vec_short.vec  bak_wiki.vec

vectors/crh/:
crh_tracked.vec  crh_vec_short.vec  crh_wiki.vec

vectors/kaz/:
kaz_tracked.vec  kaz_vec_short.vec  kaz_wiki.vec

vectors/kir/:
kir_tracked.vec  kir_vec_short.vec  kir_wiki.vec

vectors/tat/:
tat_tracked.vec  tat_vec_short.vec  tat_wiki.vec

vectors/tur/:
tur_tracked.vec  tur_vec_short.vec  tur_wiki.vec


now there is a folder `vectors` containing naive fasttext vectors of each language. e.g. `vectors/ita/ita_wiki.vec`


### alignment tools

#### required preinstalls 
ends up with MUSE repo cloned to MUSE folder

In [0]:
# clone and run a tool installing pytorch 0.4.1 with cuda 9.2 into colab (maybe works on any ubuntu 16)
!git clone https://gist.github.com/f7b7c7758a46da49f84bc68b47997d69.git colab_cuda_upgrader
!sed -i -e "s|apt-get install cuda|apt-get install cuda 9.2|g" colab_cuda_upgrader/pytorch041_cuda92_colab.sh
!bash colab_cuda_upgrader/pytorch041_cuda92_colab.sh

import sys

# install the proper version of torchtext
!git clone https://github.com/pytorch/text.git
!cd text
!{sys.executable} -m pip install .
!cd ..

!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local -c pytorch faiss-gpu cuda92

!git clone --single-branch --branch develop https://github.com/NIS-2018-CROSS-M/MUSE.git

!chmod +x MUSE/data/get_evaluation.sh

!sed -i -e "1s|^|import sys; sys.path.append(\"/usr/local/lib/python3.6/site-packages/\");|" MUSE/unsupervised.py
!sed -i -e "1s|^|import sys; sys.path.append(\"/usr/local/lib/python3.6/site-packages/\");|" MUSE/supervised.py


Cloning into 'colab_cuda_upgrader'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
--2019-02-17 16:35:41--  https://developer.nvidia.com/compute/cuda/9.2/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64.deb?PJIHCjL4gO7KwNR8LXsLuW-fr2v-Wbg2UTfI9HhfqtzZ3d_7-SBDsJTEgz-hekqKCSFw1Y86P7NuyD1CLkvXdG2YuuULt8dB1Phr8ztmQcYWe8JTVTGv6E01GsBRTtUko5SZEGg80T0jwKFTsddcBPKdXdn1egNC8lHvgXFfs7RNUONgXPuQ2j-id1o_wy3nK6q8XjfTObGaganrtONWHA [following]
--2019-02-17 16:35:41--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod

#### align embeddings
ends up with aligned embeddings stored in aligned/debug/exp0/

In [0]:
!mkdir aligned


src_lang_vd_code = langpair_to_learn_pair[0]
tgt_lang_vd_code = langpair_to_learn_pair[1]
src_lang_ft_code = vardial2fasttext[src_lang_vd_code]
tgt_lang_ft_code = vardial2fasttext[tgt_lang_vd_code]

In [0]:
!{sys.executable} MUSE/unsupervised.py --src_lang {src_lang_ft_code} --tgt_lang {tgt_lang_ft_code} \
 --src_emb vectors/{src_lang_vd_code}/{src_lang_vd_code}_wiki.vec --tgt_emb vectors/{tgt_lang_vd_code}/{tgt_lang_vd_code}_wiki.vec \
 --n_epochs 15 --epoch_size 1000000 \
 --export txt \
 --normalize_embeddings center \
 --exp_path aligned --exp_id exp0

INFO - 02/17/19 16:44:23 - 0:00:00 - ============ Initialized logger ============
INFO - 02/17/19 16:44:23 - 0:00:00 - adversarial: True
                                     batch_size: 32
                                     cuda: True
                                     dico_build: S2T
                                     dico_eval: 
                                     dico_max_rank: 15000
                                     dico_max_size: 0
                                     dico_method: csls_knn_10
                                     dico_min_size: 0
                                     dico_threshold: 0
                                     dis_clip_weights: 0
                                     dis_dropout: 0.0
                                     dis_hid_dim: 2048
                                     dis_input_dropout: 0.1
                                     dis_lambda: 1
                                     dis_layers: 2
                                     dis_most_freq

In [0]:
!mv aligned/debug/exp0/vectors-{src_lang_ft_code}.txt aligned/debug/exp0/vectors-{src_lang_vd_code}.vec
!mv aligned/debug/exp0/vectors-{tgt_lang_ft_code}.txt aligned/debug/exp0/vectors-{tgt_lang_vd_code}.vec
!ls aligned/debug/exp0/

### upload embeddings to MEGA.nz

In [0]:
!mega-put aligned/debug/exp0/vectors-{src_lang_vd_code}.vec -c /aligned-embeddings/{langpair_to_learn}/

In [0]:
!mega-put aligned/debug/exp0/vectors-{tgt_lang_vd_code}.vec -c /aligned-embeddings/{langpair_to_learn}/

# sandbox